CUSUM

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

/Users/beilakaliev/Library/Caches/pypoetry/virtualenvs/ml-modeling-TB8BmMSm-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ==============================================================================
# РЕАЛИЗАЦИЯ CUSUM ФИЛЬТРА
# ==============================================================================

def get_cusum_events(close_prices, threshold):
    """
    Применяет двусторонний CUSUM-фильтр для обнаружения значимых движений.

    :param close_prices: pd.Series с ценами закрытия.
    :param threshold: Порог (h), при превышении которого генерируется событие.
    :return: pd.Series с метками событий (1 для up, -1 для down, 0 для отсутствия события).
    """
    # Рассчитываем дневные доходности
    returns = close_prices.pct_change().dropna()
    
    s_up = 0
    s_down = 0
    events = pd.Series(index=returns.index, data=0)

    for timestamp, ret in returns.items():
        # Накопление для верхнего фильтра
        s_up = max(0, s_up + ret)
        
        # Накопление для нижнего фильтра
        s_down = min(0, s_down + ret)
        
        # Проверка порогов
        if s_up > threshold:
            events.loc[timestamp] = 1
            # Сброс обоих счетчиков после события
            s_up = 0
            s_down = 0
        elif s_down < -threshold:
            events.loc[timestamp] = -1
            # Сброс обоих счетчиков после события
            s_up = 0
            s_down = 0
            
    return events



In [3]:
# ==============================================================================
# ОСНОВНОЙ СКРИПТ
# ==============================================================================

data_folder = "../data/"
features_data_filename = 'moex_with_features.csv'
output_filename = 'moex_with_cusum_events.csv'

# --- КЛЮЧЕВОЙ ПАРАМЕТР: ПОРОГ ДЛЯ CUSUM ---
# Этот порог (h) нужно подбирать. Он представляет собой суммарную доходность.
# Например, 0.005 означает, что мы ищем накопленное движение в 0.5%.

CUSUM_THRESHOLD = 0.009

# --- ШАГ 1: Загрузка данных с признаками ---
print(f"Загрузка данных из: {os.path.join(data_folder, features_data_filename)}")
try:
    df = pd.read_csv(os.path.join(data_folder, features_data_filename))
    df['Date'] = pd.to_datetime(df['Date'])
    print("Данные успешно загружены.")
except FileNotFoundError:
    print(f"ОШИБКА: Файл не найден. Убедитесь, что скрипт feature_engineering.py был запущен.")
    exit()

# --- ШАГ 2: Применение CUSUM фильтра для каждого тикера ---
print(f"Применение CUSUM фильтра с порогом {CUSUM_THRESHOLD:.2%}...")

all_events = []
# Группируем по тикеру и применяем функцию
for ticker, group in tqdm(df.groupby('Ticker'), desc="Обработка тикеров"):
    # Важно отсортировать данные по дате внутри группы
    group = group.sort_values('Date')
    events = get_cusum_events(group['Close'], threshold=CUSUM_THRESHOLD)
    # Добавляем имя колонки для последующего объединения
    all_events.append(events.rename('cusum_event'))

# Объединяем результаты всех тикеров в одну серию
if not all_events:
    print("Не удалось сгенерировать CUSUM события.")
    exit()
    
cusum_series = pd.concat(all_events)

# Присоединяем новую колонку к исходному DataFrame
df = df.join(cusum_series)

# Заполняем пропуски (первые строки, где доходность нельзя посчитать) нулями
df['cusum_event'].fillna(0, inplace=True)
# Приводим к целочисленному типу для аккуратности
df['cusum_event'] = df['cusum_event'].astype(int)

# --- ШАГ 3: Анализ и сохранение результата ---

# Посчитаем, сколько событий мы нашли
event_counts = df['cusum_event'].value_counts()

print("\n--- Результаты CUSUM фильтрации ---")
print("Распределение меток:")
print(event_counts)

total_rows = len(df)
total_events = event_counts.get(1, 0) + event_counts.get(-1, 0)

if total_events > 0:
    print(f"\nВсего найдено {total_events} событий на {total_rows} строк данных.")
    print(f"События составляют {total_events / total_rows:.2%} от всего набора данных.")
else:
    print("\nНи одного CUSUM события не найдено. Попробуйте уменьшить CUSUM_THRESHOLD.")

# Сохраняем DataFrame с новой колонкой
output_path = os.path.join(data_folder, output_filename)
print(f"\nСохранение данных с CUSUM метками в файл: {output_path}")
df.to_csv(output_path, index=False)

print("Скрипт выполнен успешно!")

Загрузка данных из: ../data/moex_with_features.csv
Данные успешно загружены.
Применение CUSUM фильтра с порогом 0.90%...


Обработка тикеров: 100%|██████████| 77/77 [00:00<00:00, 237.13it/s]
/var/folders/lh/9k36l4vn35v4qnj9z669pmtc0000gn/T/ipykernel_88440/932637793.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['cusum_event'].fillna(0, inplace=True)



--- Результаты CUSUM фильтрации ---
Распределение меток:
cusum_event
 1    13599
-1    13166
 0     8010
Name: count, dtype: int64

Всего найдено 26765 событий на 34775 строк данных.
События составляют 76.97% от всего набора данных.

Сохранение данных с CUSUM метками в файл: ../data/moex_with_cusum_events.csv
Скрипт выполнен успешно!
